In [ ]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

In [ ]:
import cvxpy as cp
import numpy as np
from gcspy import GraphOfConvexSets

In [ ]:
inpos = np.array([
    (45, 7),
    (50, 8),
    (47, 5),
    (43, 10),
    (48, 11),
    (47, 9),
    (52, 4),
    (41, 6),
    (48, 7),
    (46, 11),
    (42, 8),
    (44, 4),
    (52, 8),
])
N = len(inpos)
posmin = np.min(inpos, axis=0)
posmax = np.max(inpos, axis=0)

In [ ]:
gcs = GraphOfConvexSets()

s = gcs.add_vertex("party")
xs = s.add_variable(2)
s.add_constraint(xs == inpos[0])

for i in range(1, N):
    v = gcs.add_vertex(f"guest {i}")
    x = v.add_variable(2)
    if i % 2 == 1:
        free = 0
        fixed = 1
    else:
        fixed = 0
        free = 1
    v.add_constraint(x[fixed] == inpos[i, fixed])
    v.add_constraint(x[free] >= posmin[free])
    v.add_constraint(x[free] <= posmax[free])
    v.add_cost(cp.norm(x[free] - inpos[i, free], 1))

In [ ]:
for tail in gcs.vertices:
    for head in gcs.vertices:
        if tail != head:
            edge = gcs.add_edge(tail, head)
            edge.add_cost(cp.norm(tail.variables[0] - head.variables[0], 1))

In [ ]:
gcs.graphviz()

In [ ]:
prob = gcs.solve_traveling_salesman()
print('Problem status:', prob.status)
print('Optimal value:', prob.value)

In [ ]:
arrowstyle = "->, head_width=3, head_length=8"
def l1_arrow(tail, head, color):
    options = dict(color=color, zorder=2, arrowstyle=arrowstyle)
    if not np.isclose(tail[0], head[0]):
        arrow = patches.FancyArrowPatch(tail, (head[0], tail[1]), **options)
        plt.gca().add_patch(arrow)
    if not np.isclose(tail[1], head[1]):
        arrow = patches.FancyArrowPatch((head[0], tail[1]), head, **options)
        plt.gca().add_patch(arrow)
    
def taxi_path(tol=1e-4):
    for edge in gcs.edges:
        if edge.y.value is not None and edge.y.value > tol:
            tail = edge.tail.variables[0].value
            head = edge.head.variables[0].value
            l1_arrow(tail, head, 'blue')

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
plt.figure()
taxi_path()
for i, v in enumerate(gcs.vertices):
    if i > 0:
        if i % 2 == 1:
            ha = 'center'
            va = 'bottom'
        else:
            ha = 'left'
            va = 'center'
        xv = v.variables[0].value
        plt.scatter(*xv, fc='g', ec='k', zorder=3)
        plt.scatter(*inpos[i], fc='w', ec='k', zorder=3, s=200)
        plt.text(*inpos[i], str(i),  ha='center', va='center', zorder=3)
        l1_arrow(inpos[i], xv, 'k')
        
plt.scatter(*inpos[0], marker='*', fc='yellow', ec='k', zorder=3, s=200, label='party')
plt.scatter(np.nan, np.nan, fc='w', ec='k', s=200, label='guest initial')
plt.scatter(np.nan, np.nan, fc='g', ec='k', label='guest optimal')
plt.plot([np.nan, np.nan], [np.nan, np.nan], c='k', label='guest motion')
plt.plot([np.nan, np.nan], [np.nan, np.nan], c='b', label='host motion')

plt.xticks(range(posmin[0] - 1, posmax[0] + 2))
plt.yticks(range(posmin[1] - 1, posmax[1] + 2))
plt.xlim([posmin[0] - 1, posmax[0] + 1])
plt.ylim([posmin[1] - 1, posmax[1] + 1])
plt.grid()
plt.legend(loc=1)

plt.savefig('party.pdf', bbox_inches='tight')